In [ ]:
import pprint
import json
import pandas as pd
pd.options.display.max_columns = 32

import vaep.pandas
import vaep.utils

In [ ]:
rawfile_metadata = 'rawfile_metadata.json'
rawfile_metadata

In [ ]:
with open(rawfile_metadata) as f:
    data = json.load(f)

key_sampled = vaep.utils.sample_iterable(data, 1)[0]
sample = data[key_sampled]
pprint.pprint(sample)

In [ ]:
sample.keys()

- essentially the all data is a dictionary with for keys: `['accession', 'cvLabel', 'name', 'value']`
- pick for each type of entry in  `['FileProperties', 'InstrumentProperties', 'MsData', 'ScanSettings', 'SampleData']` 
    - the `name` and `value` for a single in a list of entries

In [ ]:
parsed = {}
for sample_id, meta_json in data.items():
    selected = {}
    for k, entries in meta_json.items():
        for entry in entries:            
            selected[k, entry['name']] = entry['value']
    parsed[sample_id] = selected


df = pd.DataFrame.from_dict(parsed, orient='index')
df.describe()
    

In [ ]:
df